In [1]:
!pip install transformers
!pip install datasets
!pip3 install emoji==0.6.0

In [33]:
import pandas as pd
import numpy as np
import torch
from torch import cuda
import random
import os
import torch
from torch import nn
from transformers import Trainer
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
import datasets
from datasets import Dataset, load_metric
import transformers
from transformers import AutoTokenizer, AutoModel
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
train_path = '/content/drive/MyDrive/Colab Notebooks/nlp-getting-started/train.csv'
train_data = pd.read_csv(train_path)
train_data = train_data[['text', 'target']]
train_data.rename(columns={"target": "label"}, inplace=True)

test_path = '/content/drive/MyDrive/Colab Notebooks/nlp-getting-started/train.csv'
test_data = pd.read_csv(test_path)
test_data = test_data[['text']]
test_data.rename(columns={"target": "label"}, inplace=True)

train_data.head()

,text,label
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [36]:
train_data, dev_data = train_test_split(train_data, test_size=0.1, shuffle=True, stratify=train_data['label'])

print("Train dataset labels count = ", Counter(train_data['label']))
print("Dev dataset labels count = ", Counter(dev_data['label']))

Train dataset labels count =  Counter({0: 3907, 1: 2944})
Dev dataset labels count =  Counter({0: 435, 1: 327})


In [37]:
# BERTweet provides this pre-processing step by enabling the normalization argument of its tokenizer. 
# Therefore, there is no seperate pre-processing code for this model
bertweet = "vinai/bertweet-base"
tokenizer = AutoTokenizer.from_pretrained(bertweet, use_fast=False, normalization=True)
model = AutoModelForSequenceClassification.from_pretrained(
        bertweet,
        num_labels = 2,  
        output_attentions = False, 
        output_hidden_states = False,
    )

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/vinai/bertweet-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/356366feedcea0917e30f7f235e1e062ffc2d28138445d5672a184be756c8686.a2b6026e688d1b19cebc0981d8f3a5b1668eabfda55b2c42049d5eac0bc8cb2d
Model config RobertaConfig {
  "_name_or_path": "vinai/bertweet-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 130,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "

In [38]:
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True)

In [39]:
train_data = Dataset.from_pandas(train_data)
dev_data = Dataset.from_pandas(dev_data)
test_data = Dataset.from_pandas(test_data)

encoded_dataset_train = train_data.map(preprocess_function, batched=True)
encoded_dataset_dev = dev_data.map(preprocess_function, batched=True)
encoded_dataset_test = test_data.map(preprocess_function, batched=True)

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

In [40]:
columns_to_return = ['input_ids', 'label', 'attention_mask']
columns_to_return_test = ['input_ids', 'attention_mask']
encoded_dataset_train.set_format(columns=columns_to_return)
encoded_dataset_dev.set_format(columns=columns_to_return)
encoded_dataset_test.set_format(columns=columns_to_return_test)

In [41]:
batch_size = 32
metric_name = "f1"
model_name = bertweet.split("/")[-1]
task = 'tweet'

args = TrainingArguments(
    f"./save_model/{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=False,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [42]:
metric = load_metric('f1')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels, average='macro')

In [43]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset_train,
    eval_dataset=encoded_dataset_dev,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [44]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 6851
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 645


Epoch,Training Loss,Validation Loss,F1
1,No log,0.378498,0.858354
2,No log,0.378244,0.846273
3,0.410600,0.366072,0.856190


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 762
  Batch size = 32
Saving model checkpoint to ./save_model/bertweet-base-finetuned-tweet/checkpoint-215
Configuration saved in ./save_model/bertweet-base-finetuned-tweet/checkpoint-215/config.json
Model weights saved in ./save_model/bertweet-base-finetuned-tweet/checkpoint-215/pytorch_model.bin
tokenizer config file saved in ./save_model/bertweet-base-finetuned-tweet/checkpoint-215/tokenizer_config.json
Special tokens file saved in ./save_model/bertweet-base-finetuned-tweet/checkpoint-215/special_tokens_map.json
added tokens file saved in ./save_model/bertweet-base-finetuned-tweet/checkpoint-215/added_tokens.json
The following columns in t

TrainOutput(global_step=645, training_loss=0.3927707879118217, metrics={'train_runtime': 190.3657, 'train_samples_per_second': 107.966, 'train_steps_per_second': 3.388, 'total_flos': 398892290477940.0, 'train_loss': 0.3927707879118217, 'epoch': 3.0})

In [45]:
#get_test_predictions
predictions = trainer.predict(encoded_dataset_test)
preds = np.argmax(predictions.predictions, axis=-1)
#print the top 100 examples
for i in range(100):
    print(encoded_dataset_test['text'][i], preds[i], sep='\t')

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 7613
  Batch size = 32


Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all	1
Forest fire near La Ronge Sask. Canada	1
All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected	1
13,000 people receive #wildfires evacuation orders in California 	1
Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school 	1
#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires	1
#flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas	1
I'm on top of the hill and I can see a fire in the woods...	1
There's an emergency evacuation happening now in the building across the street	1
I'm afraid that the tornado is coming to our area...	1
Three people died from the heat wave so far	1
Haha South Tampa is getting flooded hah- WAIT A SECOND I LIVE IN SOUTH TAMPA WHAT AM I GONNA DO WHAT AM I GONNA DO FVCK #flooding	1
#rai

### Optional: custom class weight

In [46]:
train_labels = encoded_dataset_train['label']
print(np.bincount(train_labels))
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_labels), y=list(train_labels))
print(class_weights)

[3907 2944]
[0.87675966 1.16355299]


In [47]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        #print(inputs)
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 2 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=torch.Tensor(class_weights).to('cuda'))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [48]:
trainer = CustomTrainer(
    model,
    args,
    train_dataset=encoded_dataset_train,
    eval_dataset=encoded_dataset_dev,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [49]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 6851
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 645


Epoch,Training Loss,Validation Loss,F1
1,No log,0.419771,0.830508
2,No log,0.395365,0.845615
3,0.329200,0.395179,0.842854


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 762
  Batch size = 32
Saving model checkpoint to ./save_model/bertweet-base-finetuned-tweet/checkpoint-215
Configuration saved in ./save_model/bertweet-base-finetuned-tweet/checkpoint-215/config.json
Model weights saved in ./save_model/bertweet-base-finetuned-tweet/checkpoint-215/pytorch_model.bin
tokenizer config file saved in ./save_model/bertweet-base-finetuned-tweet/checkpoint-215/tokenizer_config.json
Special tokens file saved in ./save_model/bertweet-base-finetuned-tweet/checkpoint-215/special_tokens_map.json
added tokens file saved in ./save_model/bertweet-base-finetuned-tweet/checkpoint-215/added_tokens.json
The following columns in t

TrainOutput(global_step=645, training_loss=0.32201329904009207, metrics={'train_runtime': 188.4475, 'train_samples_per_second': 109.065, 'train_steps_per_second': 3.423, 'total_flos': 398892290477940.0, 'train_loss': 0.32201329904009207, 'epoch': 3.0})

In [50]:
#get_test_predictions
predictions = trainer.predict(encoded_dataset_test)
preds = np.argmax(predictions.predictions, axis=-1)
#print the top 100 examples
for i in range(100):
    print(encoded_dataset_test['text'][i], preds[i], sep='\t')

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 7613
  Batch size = 32


Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all	1
Forest fire near La Ronge Sask. Canada	1
All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected	1
13,000 people receive #wildfires evacuation orders in California 	1
Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school 	1
#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires	1
#flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas	1
I'm on top of the hill and I can see a fire in the woods...	1
There's an emergency evacuation happening now in the building across the street	1
I'm afraid that the tornado is coming to our area...	1
Three people died from the heat wave so far	1
Haha South Tampa is getting flooded hah- WAIT A SECOND I LIVE IN SOUTH TAMPA WHAT AM I GONNA DO WHAT AM I GONNA DO FVCK #flooding	1
#rai